# Import Data preparation

In [2]:
import logging
from abc import ABC, abstractmethod
import numpy as np 
import pandas as pd 
from typing import Union, Tuple
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Kinda useless ngl ._.
# logging.basicConfig(filename='preprocessing.log',
#                     filemode='a',
#                     format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
#                     datefmt='%H:%M:%S',
#                     level=logging.DEBUG)

logger = logging.getLogger(__name__)


# Abstract class is a strategy for handling data
class DataStrategy(ABC):
    """Abstract class defining strategy for handling data

    Args:
        ABC (_type_): _description_
    """
    @abstractmethod
    def handle_data(self, data: pd.DataFrame) -> Union[pd.DataFrame, pd.Series]:
        pass 
    
    
class DataPreProcessStrategy(DataStrategy):
    """Inherit the datastrategy and overwrite the handle_data method provided by the DataStrategy above"""
    
    def handle_data(self, data: pd.DataFrame) -> pd.DataFrame: 
        """Preprocess the dataframe

        Args:
            data (pd.DataFrame): DataFrame that need to be preprocessed.
        """
        # Drop useless coloumns
        logger.info("Begin to preprocessing the dataframe ...")
        try: 
            logger.info("1. Start dropping useless columns")
            data = data.drop(columns=[
                "Account length", 
                "State", 
                "Area code"
            ])
            logger.info("Delete useless columns complete")
        except Exception as e:
            logger.exception(f"Encounting an exception when dropping columns")
            raise e
        
        # Convert data type.
        try: 
            # Converting object column to category
            logging.info("2. Converting data to it correct data types")
            for i in data.select_dtypes(include='object').columns.to_list(): 
                data[i] = data[i].astype('category')
            # Converting target column to category 
            data['Churn'] = data['Churn'].astype('category')
            logging.info("Converting data type complete")
        except Exception as e:
            logger.exception(f"Encounting an exception when convert data type")
            raise e
        
        # Handling null value.
        try: 
            if data.isnull().sum().any()==True:
                logging.info ("3. Handling null values")   
                for i in data.select_dtypes(include=['int64', 'float64']).columns.to_list():
                    data[i].fillna(data[i].mean(), inplace=True)
                data = data.dropna()
            else: 
                logging.info("3. The data had no missing values")
        except Exception as e: 
            logger.exception(f"Encounting an exception when handling null value")
        
        # Scale if needed. 
        # Identify numerical columns
        logging.info('4. Encoding the values')
        num_col = data.select_dtypes(include=['int64', 'float64']).columns.values.tolist()

        # Identify categorical columns 
        cat_col = data.select_dtypes(include='category').columns.values.tolist()

        # Encoding the data
        numeric_transformer = Pipeline(
            steps=[("Scaler", StandardScaler())]
        )

        categorical_transformer = Pipeline(
            steps=[('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'))]
        )

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, num_col),
                ('cat', categorical_transformer, cat_col)
            ]
        )

        encoded_data = preprocessor.fit_transform(data)
        
        return encoded_data
    

class DataDivideStrategy(DataStrategy): 
    """Split the data into the dataframes for training and testing process. 

    Args:
        Dataframe (pd.DataFrame): Take in the already encoded dataframe
    """
    
    def handle_data(self, data) -> Tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
        data = pd.DataFrame(data)
        X = data.iloc[:, :-2]  # All columns except the last two
        y = data.iloc[:, -2:]  # The last two columns

        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

        return X_train, y_train, X_valid, y_valid
    
class DataCleaning(DataStrategy):
    """
    Data cleaning class which preprocesses the data and divides it into train and test data.
    """

    def __init__(self, data: pd.DataFrame, strategy: DataStrategy) -> None:
        """Initializes the DataCleaning class with a specific strategy."""
        self.df = data
        self.strategy = strategy

    def handle_data(self):
        """Handle data based on the provided strategy"""
        return self.strategy.handle_data(self.df)
    


In [4]:
import pandas as pd 
import numpy as np  
import logging 
from zenml import step 
from typing import Tuple, Annotated

@step()
def clean_data(df: pd.DataFrame) -> Tuple[
    Annotated[pd.DataFrame, 'X_train'],
    Annotated[pd.DataFrame, 'X_valid'],
    Annotated[pd.DataFrame, 'y_train'],
    Annotated[pd.DataFrame, 'y_valid']
]:
    try: 
        process_strategy = DataPreProcessStrategy()
        data_cleaning = DataCleaning(df, process_strategy)
        cleaned_data = data_cleaning.handle_data()
        
        divide_strategy = DataDivideStrategy()
        data_dividing = DataCleaning(cleaned_data, divide_strategy)
        dataframe = data_dividing.handle_data()
        return dataframe
    except Exception as e:
        logging.error(f"Error cleaning data: {e}")
        raise e 


In [11]:
# Read and transform the data
dataframe = pd.read_csv('../data/raw/telecom_churn.csv')
X_train, y_train, X_test, y_test = clean_data(dataframe)

Begin to preprocessing the dataframe ...
1. Start dropping useless columns
Delete useless columns complete
2. Converting data to it correct data types
Converting data type complete
3. The data had no missing values
4. Encoding the values


# Import Model class

In [8]:
from abc import ABC, abstractmethod
import logging
import optuna
from sklearn.base import ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

logger = logging.getLogger(__name__)

class Model(ABC): 
    def __init__(self, *args, **kwargs): 
        pass 
    
    @abstractmethod
    def fit(self, X, y): 
        pass 
    
    @abstractmethod
    def predict(self, X): 
        pass 
    
    @abstractmethod
    def optimize(self, trial: optuna.Trial, X_train, y_train, X_test, y_test): 
        """Optimize the hyperparameter of the model[]

        Args:
            trial (Optuna Trial object): Optuna Trial object
            X_train (ndarray): Training feature
            y_train (ndarray): Training target
            X_test (ndarray): Testing feature
            y_test (ndarray): Testing target
            
        """
        pass


class LogisticRegressionClassification(Model): 
    def __init__(self, *args, **kwargs): 
        super().__init__(*args, **kwargs)
        self.model = LogisticRegression(*args, **kwargs)
    
    def fit(self, X, y): 
        try: 
            self.model.fit(X, y)
            logger.info(f"Model {self.__class__.__name__} trained successfully")
            return self.model
        except ValueError as e: 
            logger.error(f"Error: {e}")
            return None
    
    def predict(self, X): 
        return self.model.predict(X)
    
    def optimize(self, trial, X_train, y_train, X_test, y_test):
        return super().optimize(trial, X_train, y_train, X_test, y_test)
    

class SVM(Model): 
    def __init__(self, *args, **kwargs): 
        super().__init__(*args, **kwargs) 
        self.model = SVC(*args, **kwargs)
    
    def fit(self, X, y): 
        try:    
            self.model.fit(X, y)
            logger.info(f"Model {self.__class__.__name__} trained successfully")
            return self.model
        except ValueError as e: 
            logger.error(f"Error: {e}")
            return None
    
    def predict(self, X): 
        return self.model.predict(X)

        

class RandomForest(Model): 
    def __init__(self, *args, **kwargs): 
        super().__init__(*args, **kwargs)
        self.model = RandomForestClassifier(*args, **kwargs)
    
    def fit(self, X, y):
        try: 
            self.model.fit(X, y)
            logger.info(f"Model {self.__class__.__name__} trained successfully")
            return self.model
        except ValueError as e: 
            logger.error(f"Error: {e}")
            return None
    
    def predict(self, X): 
        return self.model.predict(X)
    
    def optimize(self, trial: optuna.Trial, X_train, y_train, X_test, y_test):
        n_estimators = trial.suggest_int("")
        
class HyperparameterTuner: 
    """
    Class for perming hyperparameter tuning using Optuna based on Model Strategy
    """
    
    def __init__(self, model: Model, X_train, y_train, X_test, y_test ):
        self.model = model 
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        
    def optimize(self, n_trials = 100): 
        study = optuna.create_study(direction='maximize')
        study.optimize(lambda trial: self.model.optimize(trial, self.X_train, self.y_train, 
                                                         self.X_test, self.y_test), 
                                                         n_trials = n_trials)
        return study.best_trial.params
        

# Test the model 

In [12]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(2666, 18) (2666, 2) (667, 18) (667, 2)


In [13]:
rfc = RandomForest(n_estimators = 50)
rfc_train = rfc.fit(X_train,y_train)

Model RandomForest trained successfully


In [14]:
rfc_train

RandomForestClassifier(n_estimators=50)